In [1]:
import re
import requests
import datetime as dt
import pandas as pd
import plotly.express as px

In [2]:
BASE_URL = f"https://api.nasdaq.com/api"

## Utils

In [3]:
def get_response(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.2 Safari/605.1.15" 
    }

    session = requests.Session()
    response = session.get(url=url, headers=headers)

    return response.json()


def get_dataframe(data):
    try:
        cols = data["headers"]
        use_cols = list(cols.keys())

        rows = data["rows"]
    except KeyError:
        rows = data["nocp"]["nocpTable"]
    except AttributeError:
         return "There are no symbols that match your search criteria."

    df = pd.DataFrame(data=rows, columns=use_cols)
    df.columns = list(cols.values())
    df.set_index(df.columns[0], inplace=True)

    return df


def snakecase_to_camelcase(s):
    return re.sub(r'_([a-z])', lambda x: x.group(1).upper(), s)


def string_to_float(s):
    return float(re.sub(r"[\$\,]+", "", s))


## Screener

In [4]:
# There are all possible filters for the Stock Screener:
# Limit: limit = 0+ (0 = all data, 1+ = rows)
# Exchange: exchange = ["nasdaq", "nyse", "amex"]
# Market Cap: marketcap = ["mega", "large", "mid", "small", "micro", "nano"]
# Analyst Rating: recommendation = ["strong_buy", "hold", "buy", "sell", "strong_sell"]
# Sector: sector = ["technology", "telecommunications", "health_care", "finance", "real_estate", "consumer_discretionary", "consumer_staples", "industrials", "basic_materials", "energy", "utilities"]
# Region: region = ["africa", "asia", "australia_and_south_pacific", "caribbean", "europe", "middle_east", "north_america", "south_america"]
# Country: country = ["argentina", "armenia", "australia", "austria", "belgium", "bermuda", "brazil", "canada", "cayman_islands", "chile", "colombia", "costa_rica", "curacao",
#                     "cyprus", "denmark", "finland", "france", "germany", "greece", "guernsey", "hong_kong", "india", "indonesia", "ireland", "isle_of_man", "israel", "italy",
#                     "japan", "jersey","luxembourg", "macau", "mexico", "monaco", "netherlands", "norway", "panama", "peru", "philippines", "puerto_rico", "russia", "singapore",
#                     "south_africa", "south_korea","spain", "sweden", "switzerland", "taiwan", "turkey", "united_kingdom", "united_states", "usa"]

# URL Example: https://api.nasdaq.com/api/screener/stocks?limit=0&exchange=nasdaq&marketcap=mega|large|mid&recommendation=strong_buy&sector=technology&region=north_america&country=united_states

In [5]:
def screener(limit=0, **kwargs):
    parameters = {
        "exchange": ["nasdaq", "nyse", "amex"],
        "marketcap": ["mega", "large", "mid", "small", "micro", "nano"],
        "recommendation": ["strong_buy", "hold", "buy", "sell", "strong_sell"],
        "sector": ["technology", "telecommunications", "health_care", "finance", "real_estate", "consumer_discretionary", "consumer_staples", "industrials", "basic_materials", "energy", "utilities"],
        "region": ["africa", "asia", "australia_and_south_pacific", "caribbean", "europe", "middle_east", "north_america", "south_america"],
        "country": ["argentina", "armenia", "australia", "austria", "belgium", "bermuda", "brazil", "canada", "cayman_islands", "chile", "colombia", "costa_rica", "curacao",
                    "cyprus", "denmark", "finland", "france", "germany", "greece", "guernsey", "hong_kong", "india", "indonesia", "ireland", "isle_of_man", "israel", "italy",
                    "japan", "jersey","luxembourg", "macau", "mexico", "monaco", "netherlands", "norway", "panama", "peru", "philippines", "puerto_rico", "russia", "singapore",
                    "south_africa", "south_korea","spain", "sweden", "switzerland", "taiwan", "turkey", "united_kingdom", "united_states", "usa"]
    }

    url = f"{BASE_URL}/screener/stocks?limit={limit}"

    for i, v in kwargs.items():
        if isinstance(v, list):
            v = "|".join(v) if all(item in parameters[i] for item in v) else ""
        url += f"&{i}={v}"

    response = get_response(url)
    data = response["data"]["table"]

    df = get_dataframe(data)

    return df


In [6]:
screener(exchange="nasdaq", marketcap=["mega", "large", "mid"], sector=["technology", "health_care", "finance"])

,Name,Last Sale,Net Change,% Change,Market Cap
Symbol,,,,,
AAPL,Apple Inc. Common Stock,$135.21,-0.73,-0.537%,"2,344,181,741,400"
MSFT,Microsoft Corporation Common Stock,$235.81,-4.54,-1.889%,"1,757,839,312,087"
GOOG,Alphabet Inc. Class C Capital Stock,$91.78,-0.38,-0.412%,"1,187,908,540,000"
GOOGL,Alphabet Inc. Class A Common Stock,$91.12,-0.17,-0.186%,"1,179,366,160,000"
NVDA,NVIDIA Corporation Common Stock,$173.77,-3.25,-1.836%,"427,474,200,000"
...,...,...,...,...,...
TRMK,Trustmark Corporation Common Stock,$33.28,-1.51,-4.34%,"2,028,544,594"
MDRX,Veradigm Inc. Common Stock,$18.45,-0.61,-3.20%,"2,015,842,867"
KRYS,"Krystal Biotech, Inc. Common Stock",$78.16,-2.55,-3.159%,"2,012,436,090"


# SingleBase

In [7]:
class SingleBase:
    def __init__(self, ticker):
        self._years = 10
        self._ticker = ticker.upper()
        self._base_url = BASE_URL
        self._total_days = 365 * self._years + 1
        self._trading_days = 252 * self._years
        self._start_date = (dt.datetime.now() - dt.timedelta(days=self._total_days)).strftime("%Y-%m-%d")


    # Company Profile
    def _get_profile(self):
        url = f"{self._base_url}/company/{self._ticker}/company-profile"

        response = get_response(url)
        data = list(response["data"].values())

        lbls = [i["label"] for i in data]
        vals = [j["value"] for j in data]

        df = pd.DataFrame(data=vals, index=lbls, columns=[self._ticker])

        return df


    # Key Data
    def _get_summary(self):
        url = f"{self._base_url}/quote/{self._ticker}/summary?assetclass=stocks"

        response = get_response(url)
        data = list(response["data"]["summaryData"].values())

        lbls = [i["label"] for i in data]
        vals = [j["value"] for j in data]

        df = pd.DataFrame(data=vals, index=lbls, columns=[self._ticker])

        return df


    # Dividend History
    def _get_dividends(self):
        url = f"{self._base_url}/quote/{self._ticker}/dividends?assetclass=stocks"

        response = get_response(url)
        data = response["data"]["dividends"]

        df = get_dataframe(data)

        return df


    # Historical Quotes (10-Years Maximum)
    def _get_quotes_history(self):
        url = f"{self._base_url}/quote/{self._ticker}/historical?assetclass=stocks&fromdate={self._start_date}&limit={self._trading_days}"

        response = get_response(url)
        data = response["data"]["tradesTable"]

        df = get_dataframe(data)

        return df


    # Chart
    def _get_chart(self, data):
        if data.empty:
            print("No data to display.")
        else:
            data = data.applymap(lambda x: string_to_float(x))
            data.index = pd.to_datetime(data.index)
            data = data.sort_index()
            chart = px.area(x=data.index, y=data["Close/Last"])

            axes_layout = {
                            "gridcolor": "rgb(240, 240, 240)",
                            "showspikes": True,
                            "spikecolor": "rgb(120, 120, 240)",
                            "spikemode": "across",
                            "spikesnap": "cursor",
                            "spikedash": "dash",
                            "spikethickness": 0.5
                            }

            chart.update_xaxes(
                axes_layout,
                title_text="Date",
                rangeselector={
                    "buttons": [
                        {"count": 1, "label": "1M", "step": "month", "stepmode": "backward"},
                        {"count": 3, "label": "3M", "step": "month", "stepmode": "backward"},
                        {"count": 6, "label": "6M", "step": "month", "stepmode": "backward"},
                        {"count": 1, "label": "YTD", "step": "year", "stepmode": "todate"},
                        {"count": 1, "label": "1Y", "step": "year", "stepmode": "backward"},
                        {"count": 3, "label": "3Y", "step": "year", "stepmode": "backward"},
                        {"count": 5, "label": "5Y", "step": "year", "stepmode": "backward"},
                        {"label": "MAX", "step": "all"}]})

            chart.update_yaxes(axes_layout, title_text="Price", tickprefix="$")

            chart.update_layout(
                plot_bgcolor="rgb(250, 250, 250)",
                hovermode="x",
                spikedistance=-1,
                hoverdistance=-1,
                showlegend=False,
                title={
                    "text": f"{self._ticker} Stock Price History",
                    "y": 0.97,
                    "x": 0.5,
                    "xanchor": "center",
                    "yanchor": "top"})

            chart.update_traces(hovertemplate="Date: %{x}<br>Price: %{y}")

            chart.show();


    # Historical NOCP (1-Year Maximum)
    def _get_nocp_history(self, timeframe=None):
        timeframe = timeframe if timeframe in ["d5", "m1", "m3", "m6"] else "y1"

        url = f"{self._base_url}/company/{self._ticker}/historical-nocp?timeframe={timeframe}"

        response = get_response(url)
        data = response["data"]

        df = get_dataframe(data)

        return df


    # Financials (Annual/Quarterly)
    def _get_financials(self, table=None, frequency=None):
        frequency = 2 if frequency == "quarterly" else 1

        url = f"{self._base_url}/company/{self._ticker}/financials?frequency={frequency}"

        response = get_response(url)

        if table == "balance_sheet":
            data = response["data"]["balanceSheetTable"]
        elif table == "cash_flow":
            data = response["data"]["cashFlowTable"]
        elif table == "financial_ratios":
            data = response["data"]["financialRatiosTable"]
        else:
            data = response["data"]["incomeStatementTable"]

        df = get_dataframe(data)

        return df


    # Earnings Per Share (Estimated vs Reported)
    def _get_eps(self):
        url = f"{self._base_url}/quote/{self._ticker}/eps"

        response = get_response(url)
        data = response["data"]["earningsPerShare"]

        lbls = [i["period"] for i in data]
        vals = [(j["consensus"], j["earnings"]) for j in data]

        df = pd.DataFrame(data=vals, index=lbls, columns=["Estimated EPS", "Reported EPS"])

        return df


    # Earnings Forecast (Yearly/Quarterly)
    def _get_earnings_forecast(self, frequency=None):
        url = f"{self._base_url}/analyst/{self._ticker}/earnings-forecast"

        response = get_response(url)
        data = response["data"]["quarterlyForecast"] if frequency == "quarterly" else response["data"]["yearlyForecast"]

        df = get_dataframe(data)

        return df


    # Price/Earnings Ratio
    def _get_pe_ratio(self):
        url = f"{self._base_url}/analyst/{self._ticker}/peg-ratio"

        response = get_response(url)
        data = response["data"]["per"]

        lbls = [i["x"] for i in data["peRatioChart"]]
        vals = [j["y"] for j in data["peRatioChart"]]

        df = pd.DataFrame(data=vals, index=lbls, columns=[data["label"]])

        return df


    # Forecast P/E Growth Rates
    def _get_pe_growth(self):
        url = f"{self._base_url}/analyst/{self._ticker}/peg-ratio"

        response = get_response(url)
        data = response["data"]["gr"]

        lbls = [f"{i['z']} {i['x']}" for i in data["peGrowthChart"]]
        vals = [j["y"] for j in data["peGrowthChart"]]

        df = pd.DataFrame(data=vals, index=lbls, columns=[data["title"]])

        return df


    # PEG Ratio
    def _get_peg_ratio(self):
        url = f"{self._base_url}/analyst/{self._ticker}/peg-ratio"

        response = get_response(url)
        data = response["data"]["pegr"]

        lbls = data["label"]
        vals = data["pegValue"]

        df = pd.DataFrame(data=vals, index=[lbls], columns=["PEG Ratio"])

        return df


    # Short Interest
    def _get_short_interest(self):
        url = f"{self._base_url}/quote/{self._ticker}/short-interest?assetClass=stocks"

        response = get_response(url)
        data = response["data"]["shortInterestTable"]

        df = get_dataframe(data)

        return df


    # Institutional Holdings
    def _get_institutional_holdings(self, limit=5000, **kwargs):
        parameters = {
            "type_": ["total", "new", "increased", "decreased", "activity", "soldout"],
            "sortColumn": ["ownerName", "date", "sharesHeld", "sharesChange", "sharesChangePCT", "marketValue"],
            "sortOrder": ["asc", "desc"]
        }

        url = f"{self._base_url}/company/{self._ticker}/institutional-holdings?limit={limit}&tableonly=true"

        for i, v in kwargs.items():
            i = snakecase_to_camelcase(i.strip('_'))
            url += f"&{i}={v}"

        response = get_response(url)
        data = response["data"]["holdingsTransactions"]["table"]

        df = get_dataframe(data)

        return df


    # Insider Activity
    def _get_insider_activity(self, limit=5000, **kwargs):
        parameters = {
            "type_": ["all", "buys", "sells"],
            "sortColumn": ["insider", "relation", "lastDate", "transactionType", "ownType", "sharesTraded", "lastPrice", "sharesHeld"],
            "sortOrder": ["asc", "desc"]
        }

        url = f"{self._base_url}/company/{self._ticker}/insider-trades?limit={limit}"

        for i, v in kwargs.items():
            i = snakecase_to_camelcase(i.strip('_'))
            url += f"&{i}={v}"

        response = get_response(url)
        data = response["data"]["transactionTable"]["table"]

        df = get_dataframe(data)

        return df


    # Revenue EPS
    def _get_revenue_eps(self):
        limit = 1
        df = pd.DataFrame()

        while True:
            url = f"{self._base_url}/company/{self._ticker}/revenue?limit={limit}"

            response = get_response(url)
            data = response["data"]["revenueTable"]

            if not data:
                break

            tmp_df = get_dataframe(data)
            df = pd.concat([df, tmp_df], axis=1)
            limit += 1

        return df


# Ticker

In [8]:
class Ticker(SingleBase):
    def __repr__(self):
        return "nasdaq.Ticker object <%s>" % self._ticker

    @property
    def profile(self):
        return self._get_profile()

    @property
    def summary(self):
        return self._get_summary()

    @property
    def dividends(self):
        return self._get_dividends()

    @property
    def quotes_history(self):
        return self._get_quotes_history()

    @property
    def chart(self):
        return self._get_chart(self._get_quotes_history())

    @property
    def nocp_history(self):
        return self._get_nocp_history()

    @property
    def income_statement_annual(self):
        return self._get_financials()

    @property
    def balance_sheet_annual(self):
        return self._get_financials(table="balance_sheet")

    @property
    def cash_flow_annual(self):
        return self._get_financials(table="cash_flow")

    @property
    def financial_ratios_annual(self):
        return self._get_financials(table="financial_ratios")

    @property
    def income_statement_quarterly(self):
        return self._get_financials(table="quarterly")

    @property
    def balance_sheet_quarterly(self):
        return self._get_financials(table="balance_sheet", frequency="quarterly")

    @property
    def cash_flow_quarterly(self):
        return self._get_financials(table="cash_flow", frequency="quarterly")

    @property
    def financial_ratios_quarterly(self):
        return self._get_financials(table="financial_ratios", frequency="quarterly")

    @property
    def eps(self):
        return self._get_eps()

    @property
    def earnings_forecast_yearly(self):
        return self._get_earnings_forecast()

    @property
    def earnings_forecast_quarterly(self):
        return self._get_earnings_forecast(frequency="quarterly")

    @property
    def pe_ratio(self):
        return self._get_pe_ratio()

    @property
    def pe_growth(self):
        return self._get_pe_growth()

    @property
    def peg_ratio(self):
        return self._get_peg_ratio()

    @property
    def short_interest(self):
        return self._get_short_interest()

    def institutional_holdings(self, **kwargs):
        """
        Inputs:
            limit: 1+ (default=5000)
            type_: total, new, increased, decreased, activity, soldout (default=total)
            sort_column: ownerName, date, sharesHeld, sharesChange, sharesChangePCT, marketValue (default=marketValue)
            sort_order: asc, desc (default=desc)
        """
        return self._get_institutional_holdings(**kwargs)

    def insider_activity(self, **kwargs):
        """
        Inputs:
            limit: 1+ (default=5000)
            type_: all, buys, sells (default=all)
            sort_column: insider, relation, lastDate, transactionType, ownType, sharesTraded, lastPrice, sharesHeld (default=lastDate)
            sort_order: asc, desc (default=desc)
        """
        return self._get_insider_activity(**kwargs)

    @property
    def revenue_eps(self):
        return self._get_revenue_eps()


In [9]:
symbol = "aapl"
company = Ticker(symbol)

In [10]:
company.quotes_history

,Close/Last,Volume,Open,High,Low
Date,,,,,
01/18/2023,$135.21,"69,672,800",$136.815,$138.61,$135.03
01/17/2023,$135.94,"63,646,630",$134.83,$137.29,$134.13
01/13/2023,$134.76,"57,809,720",$132.03,$134.92,$131.66
01/12/2023,$133.41,"71,379,650",$133.88,$134.26,$131.44
01/11/2023,$133.49,"69,458,950",$131.25,$133.51,$130.46
...,...,...,...,...,...
01/28/2013,$16.0654,"783,086,887",$15.6366,$16.1861,$15.5664
01/25/2013,$15.71,"1,206,468,837",$16.1318,$16.2939,$15.5357
01/24/2013,$16.0893,"1,457,835,377",$16.4286,$16.6332,$16.0804
